In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from alexnet_FashionMnist import FashionMnist
from alexnet_CIFAR100 import CIFAR100
from resnet_FashionMnist import resnet_FashionMnist
from googlenet_FashionMnist import Googlenet
from vgg_FashionMnist import vgg
from d2l import torch as d2l
import numpy as np
import pandas as pd
from ptflops import get_model_complexity_info
from train_layers import train_layers
from train import train_func
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
alexnet_fashionmnist = FashionMnist()
alexnet_cifar100 = CIFAR100()
resnet_fashionmnist = resnet_FashionMnist()
vgg_fashionmnist = resnet_FashionMnist()
googlenet_fashionmnist = Googlenet()

##### using ptflops to calculate the number of the flops in the model

In [3]:
with torch.cuda.device(0):
    net = alexnet_fashionmnist
    macs, params = get_model_complexity_info(net, (1, 224, 224), as_strings=True,
                                            print_per_layer_stat=True, verbose=True)
    
    # net = alexnet_cifar100
    # macs, params = get_model_complexity_info(net, (3, 224, 224), as_strings=True,
    #                                         print_per_layer_stat=True, verbose=True)
    
    # net = resnet_fashionmnist
    # macs, params = get_model_complexity_info(net, (1, 224, 224), as_strings=True,
    #                                         print_per_layer_stat=True, verbose=True)
    
    # net = vgg_fashionmnist
    # macs, params = get_model_complexity_info(net, (1, 224, 224), as_strings=True,
    #                                         print_per_layer_stat=True, verbose=True)
    
    # net = googlenet_fashionmnist
    # macs, params = get_model_complexity_info(net, (1, 224, 224), as_strings=True,
    #                                         print_per_layer_stat=True, verbose=True)
    
    if net == resnet_fashionmnist:
        print(torch.backends.mps.is_built())
        print(torch.backends.mps.is_available())
    
    print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
    print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  46.76 M, 100.000% Params, 939.85 MMac, 99.883% MACs, 
  (0): Conv2d(11.71 k, 0.025% Params, 34.15 MMac, 3.630% MACs, 1, 96, kernel_size=(11, 11), stride=(4, 4), padding=(1, 1))
  (1): ReLU(0, 0.000% Params, 279.94 KMac, 0.030% MACs, )
  (2): MaxPool2d(0, 0.000% Params, 279.94 KMac, 0.030% MACs, kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(614.66 k, 1.314% Params, 415.51 MMac, 44.158% MACs, 96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(0, 0.000% Params, 173.06 KMac, 0.018% MACs, )
  (5): MaxPool2d(0, 0.000% Params, 173.06 KMac, 0.018% MACs, kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(885.12 k, 1.893% Params, 127.46 MMac, 13.546% MACs, 256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(0, 0.000% Params, 55.3 KMac, 0.006% MACs, )
  (8): Conv2d(1.33 M, 2.839% Params, 191.16 MMac, 20.315% MACs, 384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): 

[**我们构造一个**]高度和宽度都为224的(**单通道数据，来观察每一层输出的形状**)。
它与 :numref:`fig_alexnet`中的AlexNet架构相匹配。


In [4]:
X = torch.randn(1, 1, 224, 224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([1, 96, 54, 54])
ReLU output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Conv2d output shape:	 torch.Size([1, 256, 26, 26])
ReLU output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 256, 12, 12])
ReLU output shape:	 torch.Size([1, 256, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 256, 5, 5])
Flatten output shape:	 torch.Size([1, 6400])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1,

## 读取数据集

尽管原文中AlexNet是在ImageNet上进行训练的，但本书在这里使用的是Fashion-MNIST数据集。因为即使在现代GPU上，训练ImageNet模型，同时使其收敛可能需要数小时或数天的时间。
将AlexNet直接应用于Fashion-MNIST的一个问题是，[**Fashion-MNIST图像的分辨率**]（$28 \times 28$像素）(**低于ImageNet图像。**)
为了解决这个问题，(**我们将它们增加到$224 \times 224$**)（通常来讲这不是一个明智的做法，但在这里这样做是为了有效使用AlexNet架构）。
这里需要使用`d2l.load_data_fashion_mnist`函数中的`resize`参数执行此调整。


In [5]:
batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
# print the shape of the train_iter
list_of_i = []
for i, (X, y) in enumerate(train_iter):
    list_of_i.append(i)

print('the shape of the train_iter is:', np.array(list_of_i).shape)
# print(list_of_i)
# print the first 10 batch of the train_iter
for i, (X, y) in enumerate(train_iter):
    if i < 10:
        print('the shape of the', i, 'batch of the train_iter is:', X.shape)
    else:
        break

the shape of the train_iter is: (469,)
the shape of the 0 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 1 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 2 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 3 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 4 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 5 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 6 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 7 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 8 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 9 batch of the train_iter is: torch.Size([128, 1, 224, 224])


## [**训练AlexNet**]

现在AlexNet可以开始被训练了。与 :numref:`sec_lenet`中的LeNet相比，这里的主要变化是使用更小的学习速率训练，这是因为网络更深更广、图像分辨率更高，训练卷积神经网络就更昂贵。


In [6]:
lr, num_epochs = 0.01, 10
device = d2l.try_gpu()
Time_Layers, Time_AllEpochs, TestAcc, TrainLoss, TrainAcc, TimeEpoch, Energy_AllEpochs, TrainTime, Timport= train_layers(alexnet_fashionmnist, train_iter, test_iter, num_epochs, lr, device)
# Time_AllEpochs, TestAcc, TrainLoss, TrainAcc, TimeEpoch, Energy_AllEpochs, TrainTime, Timport= train_func(alexnet, train_iter, test_iter, num_epochs, lr, device)

training on cuda:0
epoch 1
round 0
time to device 0.004738 sec
time forward 0.214961 sec
loss time 0.023040 sec
backward time 0.135780 sec
optimizer time 0.012143 sec
training time in batch 0 cost 0.39356064796447754 sec
loss 2.299622, train acc 0.101562
round 1
time to device 0.005604 sec
time forward 0.006119 sec
loss time 0.000166 sec
backward time 0.009953 sec
optimizer time 0.000927 sec
training time in batch 1 cost 0.023998737335205078 sec
loss 2.304550, train acc 0.097656
round 2
time to device 0.005079 sec
time forward 0.006027 sec
loss time 0.000152 sec
backward time 0.009814 sec
optimizer time 0.000907 sec
training time in batch 2 cost 0.022607803344726562 sec
loss 2.305057, train acc 0.091146
round 3
time to device 0.006206 sec
time forward 0.006724 sec
loss time 0.000221 sec
backward time 0.009979 sec
optimizer time 0.000984 sec
training time in batch 3 cost 0.024990081787109375 sec
loss 2.305277, train acc 0.099609
round 4
time to device 0.005959 sec
time forward 0.006668 

In [7]:
print('Forward Layers Time: \n', 
      'Conv2d time: ', Time_Layers[0,0], '\n',
      'ReLU time: ', Time_Layers[0,1], '\n',
      'MaxPool2d time: ', Time_Layers[0,2], '\n',
      'Linear time: ', Time_Layers[0,3], '\n',
      'Dropout time: ', Time_Layers[0,4], '\n',
      'Flatten time: ', Time_Layers[0,5])
print('*'*50)
print('Time_AllEpochs: \n', 
      'Time to Device time: ', Time_AllEpochs[0,0], '\n',
      'Forward time: ', Time_AllEpochs[0,1], '\n',
      'Calculate Loss time: ', Time_AllEpochs[0,2], '\n',
      'Backward time: ', Time_AllEpochs[0,3], '\n',
      'Optimize time: ', Time_AllEpochs[0,4], '\n',
      'Test time: ', Time_AllEpochs[0,5])
print('*'*50)
print('Train Time of each epoch:', TrainTime)
print('*'*50)
print('Import data to ndarray time:', Timport)
print('*'*50)
print('TestAcc:', TestAcc)
print('*'*50)
print('TrainLoss:', TrainLoss)
print('*'*50)
print('TrainAcc:', TrainAcc)
print('*'*50)
print('TimeEpoch:', TimeEpoch)
print('*'*50)
print('Energy_AllEpochs:', Energy_AllEpochs, '\n',
      'Total Energy:',np.sum(Energy_AllEpochs[0,0]), '\n',
      'The time of the first epoch:', len(Energy_AllEpochs[0,0]))

Forward Layers Time: 
 Conv2d time:  [1.95014596] 
 ReLU time:  [0.40733886] 
 MaxPool2d time:  [0.2742281] 
 Linear time:  [0.27887607] 
 Dropout time:  [0.06950712] 
 Flatten time:  [0.01573157]
**************************************************
Time_AllEpochs: 
 Time to Device time:  [2.51804757] 
 Forward time:  [2.99582767] 
 Calculate Loss time:  [0.09946442] 
 Backward time:  [4.67357254] 
 Optimize time:  [0.42974544] 
 Test time:  [2.60910726]
**************************************************
Train Time of each epoch: [11.023797750473022, 10.88378095626831, 10.913909435272217, 10.624797344207764, 10.6014564037323, 10.598337888717651, 10.585381984710693, 10.600079536437988, 10.647079467773438, 10.663145303726196]
**************************************************
Import data to ndarray time: [0.001608133316040039, 0.0017125606536865234, 0.0016753673553466797, 0.0015609264373779297, 0.0016407966613769531, 0.0016369819641113281, 0.0016407966613769531, 0.0016329288482666016, 0.00

In [8]:
import os
working_dir = os.getcwd()
print(working_dir)

data_folder = os.path.join(working_dir, 'Data/Alexnet_layer_data')
print(data_folder)

/root/GreenAI/GPU/universal
/root/GreenAI/GPU/universal/Data/Alexnet_layer_data


In [9]:
# save the data as .npy file
np.save(os.path.join(data_folder, 'Time_Layers.npy'), Time_Layers, allow_pickle=True)
np.save(os.path.join(data_folder, 'Time_AllEpochs.npy'), Time_AllEpochs, allow_pickle=True)
np.save(os.path.join(data_folder, 'TrainTime.npy'), TrainTime, allow_pickle=True)
np.save(os.path.join(data_folder, 'TestAcc.npy'), TestAcc, allow_pickle=True)
np.save(os.path.join(data_folder, 'TrainLoss.npy'), TrainLoss, allow_pickle=True)
np.save(os.path.join(data_folder, 'TrainAcc.npy'), TrainAcc, allow_pickle=True)
np.save(os.path.join(data_folder, 'Energy_AllEpochs.npy'), Energy_AllEpochs, allow_pickle=True)